In [89]:
from enum import Enum
import itertools
import copy


with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
"""

Maybe it helps to write some ideas down.
- find arrow + direction. or does direction actually not matter?
- list of coords
- use sets? (at the end)

depending on direction:
- U - (-1,0)
- R - (0,1)
- D - (1,0)
- L - (0,-1)



In [90]:
class Dirs(Enum):
    U = [-1,0]
    R = [0,1]
    D = [1,0]
    L = [0,-1]

class GuardsGuards():
    def __init__(self,txt):
        self.txt_matrix = [[x for x in xs]for xs in txt.splitlines()]
        # self.guard = (0,0)
        # self.obs = []
        # for i, row in enumerate(self.txt_matrix):
        #     for j, c in enumerate(self.txt_matrix[i]):
        #         match c:
        #             case '^':
        #                 self.guard = (i,j)
        #                 # print(txt_matrix[i][j])
        #             case '#':
        #                 self.obs.append((i,j))
        #                 # print(txt_matrix[i][j])
        # print(self.guard)
        # print(self.obs)
        self.paths = []
        self.paths_dir = []
        self.init_board()
    
    def init_board(self):
        self.guard = (0,0)
        self.obs = []
        for i, row in enumerate(self.txt_matrix):
            for j, c in enumerate(self.txt_matrix[i]):
                match c:
                    case '^':
                        self.guard = (i,j)
                        # print(txt_matrix[i][j])
                    case '#':
                        self.obs.append((i,j))
                        # print(txt_matrix[i][j])
        print(self.guard)
        print(self.obs)

    def get_full_path(self):
        onScreen = True
        while(onScreen):
            # if dir == Dirs.L: dir = Dirs.U
            # else: dir = DIR
            for dir in Dirs:
                res = self.get_path(dir)
                self.paths.append(res[0]) # path
                self.paths_dir.append(dir)
                if res[1]: 
                    print("bye!")
                    onScreen = False
                    break

    def get_path(self, dir, check_loop=False): # guard pos
        # found = False
        # gone = False
        path = []
        path.append(self.guard)
        while(True):
            dx = self.guard[0]+dir.value[0]
            dy = self.guard[1]+dir.value[1]
            if dx < 0 or dx >= len(self.txt_matrix[0]) or dy <0 or dy >= len(self.txt_matrix): # gone
                return [path,True]
            if self.txt_matrix[dx][dy] == '#': # obs
                return [path,False]
            else:
                self.guard = (dx,dy)
                path.append(self.guard)
    
    def get_unique_positions(self):
        # print(self.paths)
        return set(itertools.chain.from_iterable(self.paths))
    
    def create_loop(self):
        counter =0
        for pos in self.get_unique_positions():
            new_matrix = copy.deepcopy(self.txt_matrix)
            new_matrix[pos[0],pos[1]] = '#'
            # run get_full_paths
            # if get_path with check_loop returns true, plus 1 
    
    def run_paths(self):
        onScreen = True
        while(onScreen):
            # if dir == Dirs.L: dir = Dirs.U
            # else: dir = DIR
            for dir in Dirs:
                res = self.get_path(dir,True)
                self.paths.append(res[0]) # path
                self.paths_dir.append(dir)
                # checkpoint
                if res[1]: 
                    print("bye!")
                    onScreen = False
                    break
    # def find_loop(self,coords):



t_input = GuardsGuards(input)
t_input.get_full_path()
t_input.paths
len(t_input.get_unique_positions())

(36, 81)
[(0, 18), (0, 83), (0, 92), (1, 3), (1, 15), (1, 67), (1, 108), (1, 126), (2, 35), (2, 52), (2, 54), (2, 70), (2, 120), (3, 5), (3, 12), (3, 29), (3, 99), (3, 110), (3, 119), (3, 126), (4, 92), (4, 114), (5, 21), (5, 24), (5, 25), (5, 29), (5, 54), (5, 72), (5, 96), (5, 99), (5, 109), (5, 117), (5, 121), (5, 128), (6, 12), (6, 13), (6, 50), (6, 51), (6, 70), (6, 102), (6, 107), (7, 13), (7, 26), (7, 28), (7, 31), (7, 39), (7, 51), (7, 66), (7, 82), (7, 88), (8, 20), (8, 31), (8, 32), (8, 42), (8, 52), (8, 69), (9, 4), (9, 12), (9, 77), (9, 81), (9, 83), (9, 126), (9, 128), (10, 2), (10, 17), (10, 55), (10, 66), (10, 68), (10, 73), (10, 88), (10, 107), (10, 109), (10, 122), (11, 1), (11, 9), (11, 35), (11, 60), (11, 63), (11, 69), (12, 8), (12, 12), (12, 35), (12, 47), (12, 86), (12, 107), (13, 29), (13, 44), (13, 53), (13, 54), (13, 59), (13, 80), (13, 87), (13, 108), (13, 121), (13, 128), (14, 14), (14, 29), (14, 42), (14, 66), (14, 67), (14, 72), (14, 115), (14, 125), (15, 1

4656

### Part 2:
Not quite sure how to proceed from here. Got an idea from [this reply](https://www.reddit.com/r/adventofcode/comments/1h7tovg/comment/m0o8qc3/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button) on reddit

- to enter a loop: position that you've been on + same direction
- check all obstacles?
